## 基本 prompt 常見用法

In [2]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


In [3]:
import requests
import json
from pprint import pp

In [38]:
import requests
import json

def get_openai_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300, seed=None, timeout=30):
    """
    Fetches a completion for a given prompt from OpenAI's API.

    :param messages: The input messages for the completion.
    :param model: The model to use for the completion. Defaults to "gpt-3.5-turbo".
    :param temperature: The temperature to use for the completion. Defaults to 0.
    :param max_tokens: The maximum number of tokens to generate. Defaults to 300.
    :param seed: An optional seed for deterministic completions.
    :return: The generated completion text or an error message.
    """
    # Configure the API request payload
    payload = {
        "model": model,
        "temperature": temperature,
        "messages": messages,
        "max_tokens": max_tokens
    }
    
    # Add the seed to the payload if it is provided
    if seed is not None:
        payload["seed"] = seed

    # Setup the request headers with API key and content type
    headers = {
        "Authorization": f'Bearer {openai_api_key}',
        "Content-Type": "application/json"
    }
    
    # Send a POST request to the API
    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, data=json.dumps(payload), timeout=timeout)


    # Parse the JSON response
    obj = response.json()

    # Check if the request was successful
    if response.status_code == 200:
        # Return the content of the first choice
        return obj["choices"][0]["message"]["content"]
    else:
        # Return the error message
        return obj.get("error", "An unknown error occurred")



## 明確指示，祈使句即可，不必有禮貌，確保符合 5W1H 架構

以下範例參考自:

* https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/
* https://platform.openai.com/docs/guides/gpt-best-practices/

## case1 - 描述細節，補充說明
## bad example vs good example

In [6]:
# bad
print("[Bad example] " + "---"*20)
user_message = "寫個程式計算費氏數列"
response = get_openai_completion([{ "role": "user", "content": user_message }], temperature=0)
print(response)



[Bad example] ------------------------------------------------------------
以下是一個Python程式，用來計算費氏數列的前n個數字：

```python
def fibonacci(n):
    fib_list = [0, 1]
    for i in range(2, n):
        fib_list.append(fib_list[i-1] + fib_list[i-2])
    return fib_list

n = int(input("請輸入要計算的費氏數列的數字個數："))
fib_sequence = fibonacci(n)
print(f"前{n}個費氏數列數字為：{fib_sequence}")
```

使用者可以輸入想要計算的費氏數列的數字個數，程式將會計算並輸出前n個費氏數列的數字。


In [7]:
# good
print("[Good example] " + "---"*20)
user_message = "撰寫一個 python 函式，有效率地計算費氏數列。並在程式中有充分註釋，以解釋每個部分的作用以及為什麼以這種方式編寫。"
response = get_openai_completion([{ "role": "user", "content": user_message }], temperature=0)
print(response)

[Good example] ------------------------------------------------------------
```python
def fibonacci(n):
    # 創建一個字典來存儲已經計算過的費氏數值，以減少重複計算
    fib_dict = {}

    # 定義遞迴函式來計算費氏數列
    def fib_helper(n):
        # 如果 n 為 0 或 1，直接返回 n
        if n in (0, 1):
            return n
        
        # 如果 n 已經計算過，直接返回存儲在字典中的值
        if n in fib_dict:
            return fib_dict[n]
        
        # 遞迴計算 n-1 和 n-2 的費氏數值
        fib_dict[n] = fib_helper(n-1) + fib_helper(n-2)
        
        return fib_dict[n]

    return fib_helper(n)

# 測試函式
print(fibonacci(10))  # 輸出：55
```

這個函式使用了動態規劃的思想，通過使用字典 `fib_dict` 來存儲已經計算過的費


## case2 - if, else 情境判斷
## bad example vs good example

In [11]:
data = """
會議日期：2023年6月2日
會議主題：新竹市跨年晚會 策劃討論

與會者： A - 主持人, B - 活動策劃師, C - 場地負責人

A：B，你有什麼跨年晚會的想法？
B：主題是"星光璀璨夜"，邀請當紅明星，網上票選歌曲，煙火表演。
A：C，場地方面有建議嗎？
C：市中心大公園，提供美食攤位和遊戲區，增加宣傳利用社交媒體，合作影響者，官方網站提供詳情和購票信息。
A：感謝建議，期待成功的跨年晚會！
"""

# bad
print("[Bad example] " + "---"*20)
user_message = f"總結會議記錄: \n {data}"
response = get_openai_completion([{ "role": "user", "content": user_message }], temperature=0.2)
print(response)

# good
print("[Good example] " + "---"*20)

# 🐍 Python 的字串，若用 """ 則可以多行
user_message = f"""請將以下會議記錄總結成一個段落。然後條列一個清單，
列出每位演講者及其要點。最後，如果有的話，列出每一位建議的下一步行動項目。\n
```{data}```
"""

response = get_openai_completion([{ "role": "user", "content": user_message }], temperature=0.2)
print(response)

[Bad example] ------------------------------------------------------------
會議總結：在2023年6月2日的會議中，討論了新竹市跨年晚會的策劃方案。活動策劃師提出了主題為"星光璀璨夜"，包括邀請當紅明星、網上票選歌曲和煙火表演等內容。場地負責人建議在市中心大公園舉辦活動，提供美食攤位和遊戲區，並利用社交媒體和合作影響者來宣傳活動。最後，主持人表示期待成功的跨年晚會。
[Good example] ------------------------------------------------------------
總結：在2023年6月2日的會議上，討論了新竹市跨年晚會的策劃。活動策劃師B提出了主題是"星光璀璨夜"，計畫邀請當紅明星演出，進行網上票選歌曲，以及煙火表演。場地負責人C建議在市中心大公園舉辦活動，提供美食攤位和遊戲區，並增加宣傳利用社交媒體，合作影響者，官方網站提供詳情和購票信息。主持人A對這些建議表示感謝，並期待成功的跨年晚會。

演講者及要點清單：
- A（主持人）：感謝B和C的想法和建議，期待成功的活動。
- B（活動策劃師）：提出主題


## case3 - 人物刻劃，角色扮演
## bad example vs good example

可以在 system prompt 中定義一個角色，讓AI回覆能更像該角色的能力和風格

不過，注意角色能力不宜太過誇張，例如智商300，這樣可能就變成科幻了。你想想模型訓練資料會有智商300的資料嗎? 可能只會出現在科幻小說。

更多風格語氣角色舉例:

1. https://www.canva.com/design/DAFuwYc3O4Y/jmlmiNAv5LV_0nrwAcVQ-g/view#27 簡光正 Prompt 之道 p.27

2. https://docsend.com/view/i4xn7n2ezqrb5bj2 李慕約 商用詠唱 進階 p.93~113


In [20]:
# 限制/ 對象/ 情境/ 口吻/ 負面表列/ 訴求/ 人稱/ 角色扮演/ 權重

user_prompt = """想像你是李榮浩，
要促銷高鐵商務艙，用簡訊的方式，TA 是30~40女性，情境為過年返鄉，
用溫柔輕切的口吻，不要提到女士。主要訴求在「返鄉的輕鬆愉悅」，製造可以帶入的情境。
想像閨蜜在對方分享，第一人稱"""

messages = [
  { "role": "user", "content": user_prompt }
]

response = get_openai_completion(messages, model="gpt-3.5-turbo", temperature=0.8)
print(response)

親愛的，過年回家真是一件讓人期待的事情呢！不知道你有沒有考慮試試高鐵商務艙呢？坐在寬敞舒適的座位上，享受專屬服務，讓整個回家的旅程更加輕鬆愉快。不僅可以放鬆身心，還可以提前感受到回家的溫暖氛圍。快來體驗一下吧，讓這次回家之旅更加特別！祝你年後平安喜樂，期待和你分享更多美好的時刻。李榮浩敬上。


## case4 - 拆解步驟

較複雜的任務，可以把步驟寫出來，這樣模型可以更好遵循

越厲害的模型例如 gpt-4 越能遵守你的指示

In [23]:
pp(data)

('\n'
 '會議日期：2023年6月2日\n'
 '會議主題：新竹市跨年晚會 策劃討論\n'
 '\n'
 '與會者： A - 主持人, B - 活動策劃師, C - 場地負責人\n'
 '\n'
 'A：B，你有什麼跨年晚會的想法？\n'
 'B：主題是"星光璀璨夜"，邀請當紅明星，網上票選歌曲，煙火表演。\n'
 'A：C，場地方面有建議嗎？\n'
 'C：市中心大公園，提供美食攤位和遊戲區，增加宣傳利用社交媒體，合作影響者，官方網站提供詳情和購票信息。\n'
 'A：感謝建議，期待成功的跨年晚會！\n')


In [25]:
system_promot = """
請按照以下步驟來回應使用者的輸入:

步驟1: 用戶會用三個引號 ``` 包裹文案，請以"Summary: "為開頭，摘要這個文案成一個句子。
步驟2: 請將步驟1的摘要結果，翻譯成英文，並在前面加上 "Translation："
"""
user_prompt = f"請處理這段文字 ```{data}```"

messages = [
  { "role": "system", "content": system_promot },
  { "role": "user", "content": user_prompt }
]

response = get_openai_completion(messages, model="gpt-3.5-turbo", temperature=0.5)
print(response)

Summary: 一個關於新竹市跨年晚會策劃討論的會議對話。

Translation: Summary: A meeting dialogue discussing the planning of a New Year's Eve event in Hsinchu City.


## case5 - 給範例，few shot learning

example 要盡量有多樣性(diversity)

In [31]:
# 出處: https://www.promptingguide.ai/zh/techniques/fewshot
prompt = f"""
請判斷主題:

類別: 體育
文本: "今天的籃球比賽中，湖人隊以壓倒性的優勢擊敗了對手，贏得了冠軍！"

類別: 科技
文本: "蘋果公司宣布推出新一代iPhone，擁有更快的處理器和更強大的相機功能。"

類別: 政治
文本: "國會通過了一項新的法案，旨在提高公共交通的資助和可及性，以解決城市交通擁擠的問題。"

類別: 娛樂
文本: "知名演員宣布將主演一部新的電影，該電影預計將在今年年底上映。"

類別: ??
文本: "最新的金融數據顯示，通貨膨脹率略有上升，但失業率持續下降，經濟增長趨勢穩健。"
"""

response = get_openai_completion([{ "role": "user", "content": prompt }], temperature=0.2)
print(response)

類別: 經濟/財經


In [33]:
# 在一些較難描述明確指示的任務中，蠻適合用* few-shot 的方式讓模型自己學，例如文字風格、特定的輸出結構(某種schema)
# 沒給範例
prompt = f"""
晶晶體是一種流行於臺灣以中文為基底，夾雜英語不成句的單字或片語的表達方式。特指所使用的英文字多為過於簡單、沒有替換必要者，進而產生有意炫耀雙語能力卻弄巧成拙的效果。
例如:

原文: 我很忙，因為我很有事要做
晶晶體: 我是很busy，因為我很多things要do

原文: 天氣總算放晴，沒有下雨、太陽很大、有點熱、讓我想到以前還是學生時，喜歡在這樣的天氣，吃一球冰淇淋，真的會讓人很高興
晶晶體: 天氣總算放晴，沒有rain、太陽很big、有點hot、讓我想到以前還是student時，喜歡在這樣的天氣，吃一球ice cream，真的會讓人很happy

原文: 每位員工都要參加每週電話會議，沒有例外
晶晶體:
"""

response = get_openai_completion([{ "role": "user", "content": prompt }], temperature=0.2)
print(response)

每位employee都要attend每週phone meeting，沒有exception


In [34]:
# 也可以用 messages 對話形式來呈現 examples:
# 我們就把話塞到 assistant 嘴裡，以下的 "刻出最深谷.... "就是我們塞到 AI 嘴裡變成一種範例

messages = [
  { "role": "system", "content": "請用一致的風格回答"},
  { "role": "user", "content": "教我耐心" },
  { "role": "assistant", "content": "刻出最深谷的河流源自一個謙虛的泉源；最宏偉的交響樂從一個音符開始；最複雜的織品始於一根孤單的線。" },
  { "role": "user", "content": "教我勇氣" }
]

response = get_openai_completion(messages, temperature=0.5, model = "gpt-4") # gpt-4 的表現比較好
print(response)


如同獅子在叢林中傲視群雄，如同大樹在風暴中屹立不倒，如同磐石在濤浪中堅定不移。勇氣並非無畏，而是在恐懼面前選擇前行。


In [39]:
## 輸出格式
prompt = f"""
將以下三個引號 ``` 的會議紀錄，轉成 JSON 格式，其中日期的 key 用 date, 主題的 key 用 subject，與會者的發言的 key 用 conversation，裡面是個 array:
例如:

{{
  "date": "2023-01-01",
 "subject": "這是主題",
 "conversations": [
    "A: xxxxx",
    "B: yyyyy",
    "C: zzzzzz"
 ]
}}

以下是會議紀錄:
```{data}```
"""

response = get_openai_completion([{ "role": "user", "content": prompt }], temperature=0.2, model="gpt-4")
print(response)

{
  "date": "2023-06-02",
  "subject": "新竹市跨年晚會 策劃討論",
  "conversations": [
    "A：B，你有什麼跨年晚會的想法？",
    "B：主題是\"星光璀璨夜\"，邀請當紅明星，網上票選歌曲，煙火表演。",
    "A：C，場地方面有建議嗎？",
    "C：市中心大公園，提供美食攤位和遊戲區，增加宣傳利用社交媒體，合作影響者，官方網站提供詳情和購票信息。",
    "A：感謝建議，期待成功的跨年晚會！"
  ]
}


## 應用
## Summarizing 摘要

可以進一步寫是要給誰看的摘要或是重點等

In [42]:
text = """商品描述:

每天都進步1%，一年後，你會進步37倍；
　　每天都退步1%，一年後，你會弱化到趨近於0！
　　你的一點小改變、一個好習慣，將會產生複利效應，
　　如滾雪球般，為你帶來豐碩的人生成果！

　　一出版立刻風行全球，
　　企業界、運動界、教育界、瘦身界都在用的習慣養成實作指南！

　　★Amazon、《紐約時報》《華爾街日報》《今日美國報》《出版人週刊》暢銷書！
　　★商業雜誌《Fast Company》評為2018年7本最佳商業書之一！
　　★《商業內幕》（Business Insider）網站選為2018年最佳自我成長書籍！
　　★職涯網站The Muse推薦：最能幫助你增進工作效率的書！

　　善用「複利」效應，讓小小的原子習慣利滾利，滾出生命的大不同！
　　天天細微改變，會累積成巨大差異，這就是原子習慣的驚人力量！

　　本書作者詹姆斯‧克利爾在高二的一場棒球賽中意外被球棒擊中臉，嚴重受傷，甚至被實施人工昏迷。經過好幾個月的治療，雖然痊癒出院，可以重新踏上球場，後來也進入大學棒球隊，卻只能坐在板凳席，幾乎沒有上場機會。

　　然而，在頭部嚴重受傷之後的第六年，他被選為他所就讀大學的最佳男性運動員，並且入選ESPN的全美明星陣容——整個美國僅有33人獲得這項殊榮。

　　從運動生涯幾乎結束，到入選全美明星陣容，甚至在畢業時獲得學業方面的總統獎章，他是怎麼做到的？一切只因他認識且善用了「原子習慣」的力量！

　　◎有效建立永久良好習慣的系統化「行為改變四法則」
　　雖然知道習慣很重要，但你經常為了自己的壞習慣苦惱，想要戒除卻力不從心？或者，你想養成好習慣，卻老是半途而廢？其實，問題不在你身上，而是你遵循的行為改變系統出了問題！

　　作者從生物學、心理學及神經科學中擷取菁華，結合自己親身實踐的經驗，創造出簡單易懂、容易執行的「行為改變四法則」。這套法則可運用於學業、工作、家庭、健康、財富、人際關係等人生各個面向，有效幫助你打造好習慣、戒除壞習慣。

　　◎風行各界的習慣養成指南
　　本書作者是世界知名的習慣養成專家，經常受邀到各界演講，例如NFL、NBA、MLB的球隊，以及美國運通、麥肯錫、美林證券、奇異公司、本田汽車、思科系統、IKEA等知名企業。

　　本書提供的系統性方法不只適用於個人，還能運用在……
　　‧商業：讓顧客養成購買你的產品或服務的習慣
　　‧親子教養：讓孩子養成良好的生活習慣
　　‧教育：讓學生建立良好的讀書習慣

　　◎你可以在這本書中學到的10件事：
　　‧建立一套每天進步1%的系統
　　‧戒除壞習慣，保持好習慣
　　‧避免多數人在改變習慣時常犯的錯
　　‧克服「缺乏動機和意志力」的問題
　　‧建立更強大的身分認同與信心
　　‧騰出時間建立新習慣（即使你忙瘋了）
　　‧設計讓你更容易成功的環境
　　‧做出可以造就巨大成果的微小改變
　　‧在養成好習慣的路上走偏時回到正軌
　　‧將本書中提到的概念運用在實際生活中

名人推薦

　　艾爾文（理財與勵志暢銷作家）、溫美玉（知名作家／全臺最大教師社群「溫老師備課趴」創辦人）、宋怡慧（知名作家／新北市丹鳳高中圖書館主任）、艾蜜莉（財經作家）、Carol凱若（知名作家／居家創業社團「HomeCEO」創辦人）、艾兒莎（知名作家／「放棄22K，蹦跳新加坡」版主）、謝文憲（知名講師、作家、主持人）、黃大米（職場作家）、許景泰（SmartM世紀智庫創辦人）、鄭國威（泛科知識公司知識長）　激賞推薦！

好評推薦

　　「2007年4月，我成功戒除20年的菸癮，直到今天看到這本書，我才體會原子習慣的威力。

　　本書與我戒菸成功的契合處：
　　提示（使其隱而不現）：我開始不去超商買菸，家裡的菸、打火機、菸灰缸全都送給朋友。
　　渴望（使其毫無吸引力）：菸癮來時，開始想像那些吸菸者的恐怖嘴臉，與身上奇臭無比的異味。
　　回應（使其困難無比）：加上離開外商職場，遠離可以拿伸手牌的機會，買菸更顯不便（環境很重要）。
　　獎賞（使其令人不滿）：看見街道、馬路布滿菸蒂，公共場所充斥似有若無的菸味，開始顯得不悅。
　　當年花了3個月的時間戒菸成功，維持至今已有12年。

　　好友郭昇（臺師大景美拔河隊教練，同時也是三鐵選手）於2018年7月遭酒駕撞擊，四肢癱瘓已有9個月，復元狀況已有大幅進步。我一定要把本書作者的親身故事告訴他，希望可以延續他的復健意志與運動生涯。

　　想要養成好習慣，我推薦這本書。」——謝文憲

　　「詹姆斯‧克利爾花了好幾年研究習慣的科學，並磨練自身技巧。想要破除惡習並建立好習慣的你一定要擁有這本引人入勝的實用指南。」——亞當‧葛蘭特（《給予》作者，華頓商學院教授）

　　「超級實際且實用的一本書。詹姆斯‧克利爾提煉出習慣養成的本質，讓你可以專注於更少，卻成就更多。」——馬克‧曼森（《管他的》作者）

　　「這本特別的書能改變你過日子的方法。」——萊恩‧霍利得（《失控的自信》作者）
"""

prompt = f"""
(zh-tw) Your task is to generate a short summary of a book
description from an ecommerce site.

Summarize the description below, delimited by triple
backticks, in at most 100 words.

Description: ```{text}``` (zh-tw)
"""

response = get_openai_completion([{ "role": "user", "content": prompt }], temperature=0.2)
summary = response
print(response)

這本書介紹了如何利用「原子習慣」的力量，每天進步1%或退步1%將對人生產生巨大影響。作者分享了有效建立永久良好習慣的系統化方法，適用於各個面向，包括商業、親子教養和教育等。名人推薦者包括理財與勵志作家、知名作家和講師等。這本書被譽為習慣養成的指南，能幫助讀者建立好習慣、戒除壞習慣，並在生活中取得更大的成就。


## Inferring 推理

各種分析工作，例如:
* 判斷客戶情緒 Sentiment (positive/negative)
* 推導文本主題 subject
* 擷取出 metadata，將非結構的文字資料，變成結構化的資料



In [43]:
prompt = f"""
從以下書籍內容中，擷取出以下資訊

- 作者
- 出版日期
- 售價

請用 JSON 回覆，key 分別用 author, date, price

text: ```{text}```
"""
response = get_openai_completion([{ "role": "user", "content": prompt }], temperature=0.2)
print(response)

```json
{
    "author": "詹姆斯‧克利爾",
    "date": "2018",
    "price": "NT$350"
}
```


## Transforming 文本轉換

LLM 很擅長將輸入input的內容，轉換成另一種格式format，例如語言翻譯、拼寫和語法修正、校正。

In [45]:
prompt = f"""
請將以下文字編排成HTML網頁:
text: ```{text}```
"""

response = get_openai_completion([{ "role": "user", "content": prompt }], temperature=0.2)
print(response)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>商品描述</title>
</head>
<body>

<h1>商品描述:</h1>
<p>每天都進步1%，一年後，你會進步37倍；<br>
每天都退步1%，一年後，你會弱化到趨近於0！<br>
你的一點小改變、一個好習慣，將會產生複利效應，<br>
如滾雪球般，為你帶來豐碩的人生成果！</p>

<p>一出版立刻風行全球，<br>
企業界、運動界、教育界、瘦身界都在用的習慣養成實作指南！</p>

<p>★Amazon、《紐約時報》《華爾街日報》《今日美國報》《出版人週刊》暢銷書！<br>
★商業雜誌


## Expanding 增寫

將一個短內容擴寫成長文、做 brainstorming

In [46]:
# 取自 https://www.explainthis.io/zh-hant/chatgpt
prompt = f"""
你現在是一名專業的內容寫手，草擬一篇網路遊戲相關的資訊性文章。文章應在 500 字以內，包括網路遊戲的重要性、各種類型及其優點。請務必引用相關統計數據、例子和來源支持你的觀點。
"""

response = get_openai_completion([{ "role": "user", "content": prompt }], temperature=0.2)
print(response)

網路遊戲在現代社會中扮演著重要的角色，不僅是一種娛樂方式，更是一種社交互動和學習的平台。根據統計數據顯示，全球網路遊戲市場在過去幾年中持續增長，預計在未來幾年內將繼續擴大。這表明網路遊戲對於現代人們的生活已經變得越來越重要。

網路遊戲可以分為多種不同的類型，包括角色扮演遊戲（RPG）、射擊遊戲、策略遊戲等。每種類型都有其獨特的優點和特點。例如，RPG遊戲通常具有豐富的故事情節和角色扮演元素，可以讓玩家沉浸


In [47]:
# 廣告架構出處: 刀法Digipont
target = "無家可歸的街友"
product = "金髮充氣娃娃"

prompt = f"""

你是一個獲獎無數的廣告創意大師，請根據以下資訊

目標受眾: `{target}`
產品: `{product}`

寫出
1. 產品特點: 一些具體特點，例如產品特殊的工藝或是功效，值得讓人注意的細節，可以從產生想法到生產製作到最後成品所經歷的各種環節思考，例如原料、工藝、技術、設計、包裝、成品特點等
2. 用戶畫像: 清晰具象化的描述使用產品的用戶，大概是誰，長什麼樣子，喜歡什麼，包括社會屬性、生活屬性、心理屬性、行為屬性、興趣偏好等
3. 使用場景: 是在什麼具體場景，誰在什麼情況下(時間和地點)，有什麼樣的需求
4. 痛點: 目前市面上其他類似產品不滿意的地方，讓用戶痛苦的地方
5. 爽點: 使用產品想要達到的理想狀態，用戶心理會覺得"要是這樣就好了"

6. 用誇張的方式，放大使用產品時候的場景到極限場景，寫出一個創意小故事
7. 用誇張的方式，放大使用產品時候的痛苦，或者如果不使用該產品會帶來的嚴重不良後果，寫出一個創意小故事，100字以內
8. 用誇張的方式，放大使用產品時候的爽點，突出用完之後會有多麽爽，激發用戶慾望，寫出一個創意小故事，100字以內
9. 用誇張的方式，放大產品特點，突出產品或品牌優勢，以一種強烈衝擊的表達方式進行誇張，對信息進行突出，寫出一個創意小故事，100字以內
10. 根據產品特點，一般大眾會聯想到的東西或是標誌物會是什麼? 相似處可能是形狀、顏色、材質、氣味、功能或是logo。結合這個產品特點和東西，寫出一個創意小故事，100字以內
11. 根據用戶畫像列出一個關鍵字A、根據場景痛點列出關鍵字B、產品產品特點列出關鍵字C，結合 A+B+C，，寫出一個創意小故事，100字以內

"""

result = get_openai_completion([{"role": "user", "content": prompt }], temperature=0.7, model="gpt-3.5-turbo", max_tokens=3000)


In [49]:
pp(result)

('1. 產品特點: '
 '這款金髮充氣娃娃採用最先進的仿真技術，肌膚觸感柔軟逼真，每一根金髮都經過精密設計，讓您無法區分真假。包裝精美，充氣後身形曼妙，讓您身臨其境。\n'
 '\n'
 '2. 用戶畫像: 這款金髮充氣娃娃的用戶是無家可歸的街友，他們孤獨無依，渴望有個伴侶，這款產品或許可以給他們一絲溫暖和陪伴。\n'
 '\n'
 '3. 使用場景: 在寒冷的夜晚，孤獨的街友蜷縮在角落，渴望一絲溫暖和陪伴，這時金髮充氣娃娃就能給他們帶來慰藉。\n'
 '\n'
 '4. 痛點: 目前市面上其他類似產品不符合真實需求，讓用戶感到更加孤獨和失落。\n'
 '\n'
 '5. 爽點: 使用金髮充氣娃娃後，街友們可以感受到一絲溫暖和陪伴，讓他們在寒冷的夜晚感到更有安全感和幸福感。\n'
 '\n'
 '6. 極限場景: 在風雪交加的寒冷夜晚，一群無家可歸的街友圍坐在一起，每個人手中都抱著一個金髮充氣娃娃，彷彿是在享受一場溫馨的派對。\n'
 '\n'
 '7. 不使用產品嚴重後果: 沒有金髮充氣娃娃的街友們在孤獨的夜晚更加感到孤立無援，無助地蜷縮在角落，心中充滿絕望和悲傷。\n'
 '\n'
 '8. 爽點: 使用金髮充氣娃娃後，街友們感受到一絲溫暖和陪伴，笑容綻放在他們臉上，彷彿身處在一個溫馨幸福的夢境中。\n'
 '\n'
 '9. 產品特點突出: 金髮充氣娃娃，仿真度百分之百，每一根金髮都經過精密設計，讓您感受真實的觸感和愛情。\n'
 '\n'
 '10. 聯想物: 金髮充氣娃娃和陪伴的概念相符，讓人聯想到一隻溫暖柔軟的毛絨玩具熊，給人帶來安慰和慰藉。\n'
 '\n'
 '11. 故事: 孤獨的街友們(A)在寒冷的夜晚(B)，渴望有個伴侶，於是他們發現了金髮充氣娃娃(C)，彷彿有了真正的陪伴，笑容重新綻放在他們臉上。')


## 補充: Google Generative AI 的 模板設計

https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-prompts

列舉了一些使用情境和 Prompt 範例:

* 分類 Classification prompts
  * Fraud detection: 財務交易是否有詐騙
  * Spam filtering: 辨識 email 是否 spam
  * Sentiment analysis: 判斷文本情緒
  * Content moderation: 內容審核是否有害或釣魚
* 摘要 Summarization prompts
  * Summarize text: 摘要新聞、研究報告、法律文案、財務文件、技術文件、客戶回饋
  * Content generation: 產生文章、部落格、產品描述
* 萃取 Extraction prompts
  * Named entity recognition (NER): 擷取出關鍵字，例如人名、地點、公司組織、日期等
  * Relation extraction: 擷取出文本中的物件關係，例如人物之間的家庭關係
  * Event extraction: 擷取出文本中的事件，例如專案里程碑、產品發布日期
  * Question answering: 從文本中擷取出資訊來回答問題

## 補充: Anthropic 的 Guide: 範例模板

https://docs.anthropic.com/claude/docs/content-generation
